# Haar Cascade Classifier

It includes multiple stages: feature extraction, feature selection, and training a classifier.

### 1. **Haar-like Features**

Haar-like features are rectangular patterns that detect specific visual characteristics. A common set of Haar features for `face detection` includes:

- **Edge features**: Detects boundaries, such as between the forehead and eyes.
- **Line features**: Detects horizontal or vertical changes, useful for spotting features like the bridge of the nose.
- **Four-rectangle features**: Used for detecting contrast, like between eyes and eyebrows.

Each feature calculates the difference in pixel intensity between black and white regions. This is done by summing the pixel values of the regions and subtracting the sums.

### 2. **Integral Image**

An integral image helps to compute the sum of pixel intensities within a rectangular region very quickly. It’s a 2D matrix where each element represents the sum of all pixels above and to the left of it.

- To create an integral image, use the formula:
$$
\text{IntegralImage}(x, y) = \text{Image}(x, y) + \text{IntegralImage}(x-1, y) + \text{IntegralImage}(x, y-1) - \text{IntegralImage}(x-1, y-1)
$$
- Using this integral image, we can compute the sum of any rectangular area in constant time.

### 3. **Feature Selection Using AdaBoost**

**AdaBoost** assigns higher weights to incorrectly classified samples to "boost" weak classifiers into a strong classifier. The algorithm combines many weak classifiers (simple threshold-based on a single Haar-like feature) to build a cascade of classifiers that improves detection accuracy.

### 4. **Training the Cascade Classifier**

The cascade structure consists of multiple stages. Each stage is a separate classifier that filters out negative samples (non-faces) quickly, passing only promising regions to the next stage.

In [ ]:
import numpy as np
import cv2

# Step 1: Compute the Integral Image
def integral_image(img):
    # Sum over rows and columns to get the integral image
    return img.cumsum(axis=0).cumsum(axis=1)

# Step 2: Define Haar-like features
class HaarFeature:
    def __init__(self, feature_type, position, width, height):
        self.feature_type = feature_type  # Types: 'two-rect', 'three-rect', 'four-rect'
        self.position = position          # (x, y) position
        self.width = width
        self.height = height

    def compute(self, int_img):
        x, y = self.position
        w, h = self.width, self.height

        if self.feature_type == 'two-rect':
            # Example of horizontal two-rect feature
            white = int_img[y:y+h, x:x+w//2].sum()
            black = int_img[y:y+h, x+w//2:x+w].sum()
            return white - black
        # Add more feature types as needed
        return 0

# Step 3: Create a Classifier based on Haar features
class WeakClassifier:
    def __init__(self, feature, threshold, polarity):
        self.feature = feature
        self.threshold = threshold
        self.polarity = polarity

    def predict(self, int_img):
        feature_value = self.feature.compute(int_img)
        return 1 if self.polarity * feature_value < self.polarity * self.threshold else 0

# Step 4: Train a simple weak classifier (simplified without AdaBoost)
def train_classifier(pos_imgs, neg_imgs, feature):
    best_classifier = None
    best_accuracy = 0
    for threshold in np.linspace(-1, 1, num=10):  # Test different thresholds
        for polarity in [1, -1]:
            correct = 0
            classifier = WeakClassifier(feature, threshold, polarity)
            for img in pos_imgs:
                if classifier.predict(integral_image(img)) == 1:
                    correct += 1
            for img in neg_imgs:
                if classifier.predict(integral_image(img)) == 0:
                    correct += 1
            accuracy = correct / (len(pos_imgs) + len(neg_imgs))
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_classifier = classifier
    return best_classifier

# Step 4: full ver
def adaboost_train(features, pos_integral_images, neg_integral_images):
    # Initialize weights for positive and negative samples
    num_pos, num_neg = len(pos_integral_images), len(neg_integral_images)
    weights = np.hstack((np.ones(num_pos) / (2 * num_pos), np.ones(num_neg) / (2 * num_neg)))
    classifiers = []

    for feature in features:
        best_classifier = None
        best_accuracy = 0
        
        for threshold in np.linspace(-1, 1, num=10):
            for polarity in [1, -1]:
                # Initialize a weak classifier with a feature
                classifier = WeakClassifier(feature, threshold, polarity)
                predictions = []

                # Calculate accuracy on positive and negative samples
                for img in pos_integral_images:
                    predictions.append(classifier.predict(img))
                for img in neg_integral_images:
                    predictions.append(1 - classifier.predict(img))
                
                accuracy = sum(predictions) / (num_pos + num_neg)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_classifier = classifier
        
        classifiers.append(best_classifier)
        # Update weights for misclassified samples (simplified)
        # Update weights in AdaBoost to focus on misclassified samples
    return classifiers

# Step 5: Apply a cascade of classifiers to detect faces
def detect_faces(img, classifiers):
    int_img = integral_image(img)
    for classifier in classifiers:
        if classifier.predict(int_img) == 0:
            return False  # Non-face region
    return True  # Face detected

# Step 5: full ver
class CascadeStage:
    def __init__(self, classifiers, threshold):
        self.classifiers = classifiers
        self.threshold = threshold

    def classify(self, integral_image):
        stage_sum = sum(clf.predict(integral_image) for clf in self.classifiers)
        return stage_sum >= self.threshold  # Pass if stage sum is above threshold
    
class HaarCascade:
    def __init__(self, stages):
        self.stages = stages

    def detect(self, image):
        integral_img = integral_image(image)
        for stage in self.stages:
            if not stage.classify(integral_img):
                return False  # Non-face region
        return True  # Face detected
    
# Step 6: (only for full ver) apply the trained cascade on different regions (windows) of an image to detect faces.
def sliding_window(image, cascade, window_size=(24, 24), step_size=4):
    detected_regions = []
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            window = image[y:y+window_size[1], x:x+window_size[0]]
            if cascade.detect(window):
                detected_regions.append((x, y, window_size[0], window_size[1]))
    return detected_regions

# Example Usage
if __name__ == "__main__":
    # Load example images (replace with actual images)
    pos_imgs = [cv2.imread("face1.jpg", cv2.IMREAD_GRAYSCALE)]  # Positive samples
    neg_imgs = [cv2.imread("non_face1.jpg", cv2.IMREAD_GRAYSCALE)]  # Negative samples

    # Define a Haar feature and train a classifier
    feature = HaarFeature("two-rect", (0, 0), 24, 24)
    classifier = train_classifier(pos_imgs, neg_imgs, feature)

    # Cascade of classifiers (for simplicity, using only one trained classifier)
    cascade = [classifier]

    # Detect face in a test image
    test_img = cv2.imread("test.jpg", cv2.IMREAD_GRAYSCALE)
    if detect_faces(test_img, cascade):
        print("Face detected!")
    else:
        print("No face detected.")

### Explanation of Key Components

1. **Integral Image Calculation**: This provides rapid access to rectangular regions, speeding up feature computation.
2. **Haar Feature Calculation**: In this example, we compute a two-rectangle feature by summing two areas and taking the difference.
3. **Weak Classifier Training**: Here, a single weak classifier is trained with various thresholds and polarities. Normally, multiple features and AdaBoost are used to create a strong classifier.
4. **Cascade of Classifiers**: For simplicity, we use one classifier here, but in a full Haar Cascade, several stages are used for efficient face detection.

### Limitations and Enhancements

- **AdaBoost**: A simplified version is shown, but a real implementation would iteratively adjust sample weights and use multiple features per classifier.
- **Training Time**: A real Haar Cascade requires thousands of positive and negative samples and extensive computational resources.